- Increasingly difficult Environment
- Positive reward for populating increasingly "deep" blending tanks ?
- RL for chem sched paper (https://arxiv.org/pdf/2203.00636)
- Masking (https://sb3-contrib.readthedocs.io/en/master/modules/ppo_mask.html, https://arxiv.org/pdf/2006.14171)
    - Adding binary decision variables ?g  
    - Requires discrete action space (only integer flows -> treated as categories ?)
    - masking: disable incoming flows (resp. outgoing flows) for tanks at UB inv limit (resp. LB inv. limit), disable selling/buying when available = 0
    - multiple envs with multiple agents ? (MARL, https://arxiv.org/pdf/2103.01955)
        - Predict successive pipelines ("source > blend" then "blend > blend" (as many as required) then "blend > demand")
        - Each agent has access to the whole state
        - Action mask is derived from the previous agent's actions (0 if inventory at bounds or incoming flow already reserved, else 1)
        - https://github.com/Rohan138/marl-baselines3/blob/main/marl_baselines3/independent_ppo.py
- Safe RL: (https://proceedings.mlr.press/v119/wachi20a/wachi20a.pdf)
    - "Unsafe state" ? > Do not enforce constraints strictly, instead opt for early episode termination to show which states are unsafe ? 
    - Implementations:
        - https://pypi.org/project/fast-safe-rl/#description (Policy optimizers)
        - https://github.com/PKU-Alignment/safety-gymnasium/tree/main/safety_gymnasium (environments; "cost" ?)


- Masking: Discretization of action space is too slow/might not work -> Need to implement masking for continuous action space
- Recurrent policy makes the most sense ? (window of demand forecasts)
- https://www.reddit.com/r/reinforcementlearning/comments/17l5b47/invalid_action_masking_when_action_space_is/
    - Suggestion of autoregressive model for having constraints respected: one predicted action is input to a second model
    - Suggestion of editing the distribution in such a way that the constraint is respected
- https://www.sciencedirect.com/science/article/pii/S0098135420301599
    - Choice of ELU activation ?
    - Choice of NN size ?
    - "The feature engineering in the net inventory means the network does not have to learn these relationships itself, which did help speed training." ?
- Simplify the problem (remove tanks 5 to 8), find the optimal solution with Gurobi

- Proportional penalty instead of flat
- Solution pool from Gurobi for data generation
- Uniformize the distribution profile
    - Idea is to remove start/end of episode effects to make the distribution simpler (see photo)
    - -> Simulate infinite sup/dem profile
    - -> simulate env with 12 time periods, only use the first 6 for the data, then do the same by shifting by 12.
        - -> Need to implement non-zero initial inv states for both gym and gurobi

- Clarify how DT works at inference time

In [3]:
import sys, os
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))
try:
    print(curr_dir)
except:
    curr_dir = os.path.dirname(os.path.abspath(os.getcwd()))
    os.chdir(curr_dir)
    print(curr_dir)

c:\Users\adame\OneDrive\Bureau\CODE\BlendingRL


In [4]:
import json
import numpy as np
import torch as th
from stable_baselines3 import PPO, DDPG, SAC, TD3
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.policies import ActorCriticPolicy
from stable_baselines3.common.callbacks import *
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, VecCheckNan
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.utils import safe_mean

from envs import BlendEnv, flatten_and_track_mappings, reconstruct_dict
from models import *
from utils import *
from math import exp, log
import yaml
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [8]:
# Regex for tensorboard
# Gives the current day's runs of the given config list

L = []
cfg_list = range(41, 48)
for cfg in cfg_list:
    L.append("(" + str(cfg) + "_" + datetime.now().strftime('%m%d') + ")")
"|".join(L)

'(41_1111)|(42_1111)|(43_1111)|(44_1111)|(45_1111)|(46_1111)|(47_1111)'

In [7]:
L = []
cfg_list = range(41, 48)
for cfg in cfg_list:
    L.append("(/" + str(cfg) + "/)")
"|".join(L)

'(/41/)|(/42/)|(/43/)|(/44/)|(/45/)|(/46/)|(/47/)'

In [23]:
############## ENV CONFIGURATION ##############
CONFIG = 34         # See /configs
layout = "simple" # See /img
###############################################

In [24]:
with open(f"./configs/{CONFIG}.yaml", "r") as f:
    s = "".join(f.readlines())
    cfg = yaml.load(s, Loader=yaml.FullLoader)

In [25]:
if cfg["custom_softmax"]:
    policytype = CustomMLP_ACP_simplest_softmax
elif cfg["policytype"] == "MLP":
    policytype = "MlpPolicy"
elif cfg["policytype"] == "MLPtanh":
    policytype = CustomMLP_ACP_simplest_tanh
    
optimizer_cls = eval(cfg["optimizer"])

if cfg["model"]["act_fn"] == "ReLU":
    act_cls = th.nn.ReLU
elif cfg["model"]["act_fn"] == "tanh":
    act_cls = th.nn.Tanh
elif cfg["model"]["act_fn"] == "sigmoid":
    act_cls = th.nn.Sigmoid

In [26]:
connections, action_sample = get_jsons(layout)
sources, blenders, demands = get_sbp(connections)

In [27]:
T = 6
if layout == "base":
    sigma = {"s1":{"q1": 0.06}, "s2":{"q1": 0.26}}
    sigma_ub = {"p1":{"q1": 0.16}, "p2":{"q1": 1}}
    sigma_lb = {"p1":{"q1": 0}, "p2":{"q1": 0}}
else:
    sigma = {s:{"q1": 0.06} for s in sources}
    sigma_ub = {d:{"q1": 0.16} for d in demands}
    sigma_lb = {d:{"q1": 0} for d in demands}
    
s_inv_lb = {s: 0 for s in sources}
s_inv_ub = {s: 999 for s in sources}
d_inv_lb = {d: 0 for d in demands}
d_inv_ub = {d: 999 for d in demands}
betaT_d = {d: 1 for d in demands} # Price of sold products
b_inv_ub = {j: 30 for j in blenders} 
b_inv_lb = {j: 0 for j in blenders}
betaT_s = {s: cfg["env"]["product_cost"]  for s in sources} # Cost of bought products

if cfg["env"]["uniform_data"]:
    if cfg["env"]["max_pen_violations"] < 999:
        max_ep_length = 50
        tau0   = {s: [np.random.normal(20, 3) for _ in range(max_ep_length)] for s in sources}
        delta0 = {d: [np.random.normal(20, 3) for _ in range(max_ep_length)] for d in demands}
        T = max_ep_length
        
    else:
        tau0   = {s: [np.random.normal(20, 3) for _ in range(13)] for s in sources}
        delta0 = {d: [np.random.normal(20, 3) for _ in range(13)] for d in demands}
else:
    tau0   = {s: [10, 10, 10, 0, 0, 0] for s in sources}
    delta0 = {d: [0, 0, 0, 10, 10, 10] for d in demands}

In [28]:
env = BlendEnv(v = False, T = T,
               D = cfg["env"]["D"], Q = cfg["env"]["Q"], P = cfg["env"]["P"], B = cfg["env"]["B"], Z = cfg["env"]["Z"], M = cfg["env"]["M"],
               reg = cfg["env"]["reg"], reg_lambda = cfg["env"]["reg_lambda"],
               MAXFLOW = cfg["env"]["maxflow"], alpha = cfg["env"]["alpha"], beta = cfg["env"]["beta"], 
               max_pen_violations = cfg["env"]["max_pen_violations"], illeg_act_handling = cfg["env"]["illeg_act_handling"],
               connections = connections, action_sample = action_sample, 
               tau0 = tau0, delta0 = delta0, sigma = sigma,
               sigma_ub = sigma_ub, sigma_lb = sigma_lb,
               s_inv_lb = s_inv_lb, s_inv_ub = s_inv_ub,
               d_inv_lb = d_inv_lb, d_inv_ub = d_inv_ub,
               betaT_d = betaT_d, betaT_s = betaT_s,
               b_inv_ub = b_inv_ub,
               b_inv_lb = b_inv_lb)

In [29]:
env = Monitor(env)
env = DummyVecEnv([lambda: env])
env = VecNormalize(env, 
                   norm_obs=cfg["obs_normalizer"], 
                   norm_reward=cfg["reward_normalizer"])
# env = VecCheckNan(env, raise_exception=True)

In [30]:
policy_kwargs = dict(
    net_arch=[dict(pi = [cfg["model"]["arch_layersize"]] * cfg["model"]["arch_n"], 
                   vf = [cfg["model"]["arch_layersize"]] * cfg["model"]["arch_n"])],
    activation_fn = act_cls,
    log_std_init = cfg["model"]["log_std_init"]
)

In [31]:
print(policytype)

if optimizer_cls == PPO:
    kwa = dict(policy = policytype, 
                env = env,
                tensorboard_log = "./logs",
                clip_range = cfg["model"]["clip_range"],
                learning_rate = cfg["model"]["lr"] if not cfg["model"]["lr_sched"] else (lambda p: cfg["model"]["lr"] + (cfg["model"]["lr_end"] - cfg["model"]["lr"]) * p),
                ent_coef = cfg["model"]["ent_coef"],
                use_sde = cfg["model"]["use_sde"],
                batch_size = cfg["model"]["batch_size"],
                policy_kwargs = policy_kwargs)
    
else:
    kwa = dict(policy = policytype, 
                env = env,
                tensorboard_log = "./logs",
                batch_size = cfg["model"]["batch_size"],
                learning_rate = cfg["model"]["lr"])

model = optimizer_cls(**kwa)

if cfg["starting_point"]:
    try:
        cfg_start = int(cfg["starting_point"])
        bin_ = get_bin(cfg_start)
        directory = f"C:\\Users\\adame\\OneDrive\\Bureau\\CODE\\BlendingRL\\models\\{layout}\\{bin_}\\{cfg_start}"
        chosen, mod_chosen = "", 0
        for f in os.listdir(directory):
            mod_time = os.path.getmtime(os.path.join(directory, f))
            if mod_time > mod_chosen:
                chosen = os.path.join(f"models\\{layout}\\{bin_}\\{cfg_start}", f)
        model.set_parameters(chosen)
        
    except ValueError:
        model.set_parameters(cfg["starting_point"])

MlpPolicy


In [32]:
# PPO(learning_rate=lambda p: a + (b-a)* p)

If batch_size = 64 and n_steps = 2048, then 1 epoch = 2048/64 = 32 batches

In [33]:
bin_ = get_bin(cfg['id'])
entcoef = str(model.ent_coef) if type(model) == PPO else ""
cliprange = str(model.clip_range(0)) if type(model) == PPO else ""
model_name = f"models/{layout}/{bin_}/{cfg['id']}/{cfg['id']}_{datetime.now().strftime('%m%d-%H%M')}"
model_name

'models/simple/25-36/34/34_1023-2149'

In [34]:
class CustomLoggingCallbackPPO(BaseCallback):
    def __init__(self, schedule_timesteps, start_log_std=2, end_log_std=-1, std_control = cfg["clipped_std"]):
        super().__init__(verbose = 0)
        self.print_flag = False
        self.std_control = std_control
        
        self.start_log_std = start_log_std
        self.end_log_std = end_log_std
        self.schedule_timesteps = schedule_timesteps
        self.current_step = 0
        
        self.pen_M, self.pen_B, self.pen_P, self.pen_reg, self.pen_nv = [], [], [], [], []
        self.n_pen_M, self.n_pen_B, self.n_pen_P = [], [], []
        self.units_sold, self.units_bought, self.rew_sold = [], [], []
        
        
    def _on_rollout_end(self) -> None:
        self.logger.record("penalties/in_out",              sum(self.pen_M)/len(self.pen_M))
        self.logger.record("penalties/buysell_bounds",      sum(self.pen_B)/len(self.pen_B))
        self.logger.record("penalties/tank_bounds",         sum(self.pen_P)/len(self.pen_P))
        
        self.logger.record("penalties/n_in_out",            sum(self.n_pen_M)/len(self.n_pen_M))
        self.logger.record("penalties/n_buysell_bounds",    sum(self.n_pen_B)/len(self.n_pen_B))
        self.logger.record("penalties/n_tank_bounds",       sum(self.n_pen_P)/len(self.n_pen_P))
        self.logger.record("penalties/n_vltn",              sum(self.pen_nv)/len(self.pen_nv))
        
        self.logger.record("penalties/units_sold",          sum(self.units_sold)/len(self.units_sold))
        self.logger.record("penalties/units_bought",        sum(self.units_bought)/len(self.units_bought))
        self.logger.record("penalties/rew_sold",            sum(self.rew_sold)/len(self.rew_sold))
        
        print(self.current_step, ":", safe_mean([ep_info["r"] for ep_info in model.ep_info_buffer]))
        
        self.pen_M, self.pen_B, self.pen_P, self.pen_reg, self.pen_nv = [], [], [], [], []
        self.n_pen_M, self.n_pen_B, self.n_pen_P = [], [], []
        self.units_sold, self.units_bought, self.rew_sold = [], [], []
        
    def _on_step(self) -> bool:
        
        log_std: th.Tensor = self.model.policy.log_std
        t = self.locals["infos"][0]['dict_state']['t']
        
        if self.locals["infos"][0]["terminated"] or self.locals["infos"][0]["truncated"]: # record info at each episode end
            self.pen_M.append(self.locals["infos"][0]["pen_tracker"]["M"])
            self.pen_B.append(self.locals["infos"][0]["pen_tracker"]["B"])
            self.pen_P.append(self.locals["infos"][0]["pen_tracker"]["P"])
            
            self.n_pen_M.append(-self.locals["infos"][0]["pen_tracker"]["M"]/cfg["env"]["M"])
            self.n_pen_B.append(-self.locals["infos"][0]["pen_tracker"]["B"]/cfg["env"]["P"])
            self.n_pen_P.append(-self.locals["infos"][0]["pen_tracker"]["P"]/cfg["env"]["B"])
            self.pen_nv.append(self.locals["infos"][0]["pen_tracker"]["n_violations"])
            
            self.units_sold.append(self.locals["infos"][0]["pen_tracker"]["units_sold"])
            self.units_bought.append(self.locals["infos"][0]["pen_tracker"]["units_bought"])
            self.rew_sold.append(self.locals["infos"][0]["pen_tracker"]["rew_sold"])
        
        # if self.num_timesteps%2048 < 6 and t == 1: # start printing
        #     self.print_flag = True
            
        # if self.print_flag:
        #     print("\nt:", t)
        #     if np.isnan(self.locals['rewards'][0]) or np.isinf(self.locals['rewards'][0]):
        #         print(f"is invalid reward {self.locals['rewards'][0]}")
        #     for i in ['obs_tensor', 'actions', 'values', 'clipped_actions', 'new_obs', 'rewards']:
        #         if i in self.locals:
        #             print(f"{i}: " + str(self.locals[i]))
        #     if t == 6:
        #         self.print_flag = False
        #         print(f"\n\nLog-Std at step {self.num_timesteps}: {log_std.detach().cpu().numpy()}")
        #         print(self.locals["infos"][0]["pen_tracker"])
        #         print("\n\n\n\n\n")
                
        if self.std_control:
            progress = self.current_step / self.schedule_timesteps
            new_log_std = self.start_log_std + progress * (self.end_log_std - self.start_log_std)
            self.model.policy.log_std.data.fill_(new_log_std)
            self.current_step += 1
                
        return True

In [35]:
total_timesteps = 1e5
log_callback = CustomLoggingCallbackPPO(schedule_timesteps=total_timesteps) if optimizer_cls == PPO else CustomLoggingCallbackDDPG()
callback = CallbackList([log_callback])
model_name

'models/simple/25-36/34/34_1023-2149'

In [36]:
logpath = model_name[len("models/"):]
print(f"logging at {logpath}")
model.learn(total_timesteps = total_timesteps,
            progress_bar = False,
            tb_log_name = logpath,
            callback = callback,
            reset_num_timesteps = False)

logging at simple/25-36/34/34_1023-2149
2048 : -38262.387173309995
4096 : -35037.36655872
6144 : -31934.28033633
8192 : -26427.54269349
10240 : -22356.47988014
12288 : -19999.82570883
14336 : -17064.042057069997
16384 : -13799.584824749998
18432 : -12075.939521870001
20480 : -9942.88496025
22528 : -8209.231710719998
24576 : -6876.682527270002
26624 : -5369.016219349999
28672 : -4985.15545598
30720 : -3927.5333796699997
32768 : -2957.8563028500002
34816 : -2561.7407125099994
36864 : -2102.9350889700004
38912 : -1468.6648103099997
40960 : -1306.8101320300002
43008 : -1103.7874256699997
45056 : -809.20454111
47104 : -581.8742708600001
49152 : -481.5362704600001
51200 : -310.08267188
53248 : -174.79387455999998
55296 : -91.19813708
57344 : -9.051597730000003
59392 : 52.89979792
61440 : 101.94062487000001
63488 : 127.89303318
65536 : 158.61854684
67584 : 166.65592941
69632 : 179.93371213000003
71680 : 186.37560455000002
73728 : 192.88894446999998
75776 : 195.36590621000002
77824 : 200.09567

In [30]:
import re

def save_next_file(directory, model_name):
    base_pattern = re.compile(model_name + r"_(\d+)\.zip")
    
    try:
        files = os.listdir(directory)
    except:
        os.mkdir(directory)
        
        files = os.listdir(directory)
        
    max_number = 0
    for file in files:
        match = base_pattern.match(file)
        if match:
            number = int(match.group(1))
            max_number = max(max_number, number)
    
    # Generate the next filename
    next_file_number = max_number + 1
    next_file_name = f"{model_name}_{next_file_number}"
    next_file_path = os.path.join(directory, next_file_name)
    
    model.save(next_file_path)
    
save_next_file(os.path.dirname(model_name), os.path.basename(model_name) )

# Testing

In [72]:
model = PPO.load("models\\simplest\\25-36\\30\\30_0930-1359_1.zip")

In [73]:
# M,Q,P,B,Z,D = 10, 0, 5, 5, 1, 0
M, Q, P, B, Z, D  = cfg["env"]["M"], cfg["env"]["Q"], cfg["env"]["P"], cfg["env"]["B"], cfg["env"]["Z"], 0
# M,Q,P,B,Z,D = 0, 0, 0, 0, 1, 0

In [75]:
if cfg["env"]["uniform_data"]:
    tau0   = {s: [np.random.binomial(1, 0.7) * np.random.normal(15, 2) for _ in range(20)] for s in sources}
    delta0 = {d: [np.random.binomial(1, 0.7) * np.random.normal(15, 2) for _ in range(20)] for d in demands}
else:
    tau0   = {s: [10, 10, 10, 0, 0, 0] for s in sources}
    delta0 = {d: [0, 0, 0, 10, 10, 10] for d in demands}

In [76]:
env = BlendEnv(v = True, 
               D = cfg["env"]["D"], 
               Q = cfg["env"]["Q"], 
               P = cfg["env"]["P"], 
               B = cfg["env"]["B"], 
               Z = cfg["env"]["Z"], 
               M = cfg["env"]["M"],
               reg = cfg["env"]["reg"],
               reg_lambda = cfg["env"]["reg_lambda"],
               MAXFLOW = cfg["env"]["maxflow"],
               alpha = cfg["env"]["alpha"],
               beta = cfg["env"]["beta"],
               connections = connections, 
               action_sample = action_sample,
               tau0 = tau0,delta0 = delta0,
               sigma = sigma,
               sigma_ub = sigma_ub, sigma_lb = sigma_lb,
               s_inv_lb = s_inv_lb, s_inv_ub = s_inv_ub,
               d_inv_lb = d_inv_lb, d_inv_ub = d_inv_ub,
               betaT_d = betaT_d, betaT_s = betaT_s,
               b_inv_ub = b_inv_ub,
               b_inv_lb = b_inv_lb)
env = Monitor(env)

In [77]:
with th.autograd.set_detect_anomaly(True):
    obs = env.reset()
    obs, obs_dict = obs
    for k in range(env.T):
        action, _ = model.predict(obs, deterministic=False)
        print("\n\n",env.pen_tracker)
        print(action)
        print("\n\n   ",reconstruct_dict(action, env.mapping_act))
        obs, reward, done, term, _ = env.step(action)
        print(obs)
        dobs = reconstruct_dict(obs, env.mapping_obs)
        print("\n    >>     ",dobs["sources"], dobs["blenders"], dobs["demands"])
        print("   " ,reward)
        



 {'M': 0, 'B': 0, 'P': 0, 'Q': 0, 'reg': 0, 'n_violations': 0}
[ 2.2490938  0.        10.549076   0.       ]


    {'source_blend': {'s1': {'j1': 2.2490938}}, 'blend_demand': {'j1': {'p1': 0.0}}, 'tau': {'s1': 10.549076}, 'delta': {'p1': 0.0}}
Increased reward by 8.29998230934143 through tank population in s1
j1: inv: 0, in_flow_sources: 2.249093770980835, in_flow_blend: 0, out_flow_blend: 0, out_flow_demands: 0.0
Increased reward by 4.49818754196167 through tank population in j1
Increased reward by 0 through tank population in p1
[ 8.299982   2.2490938  0.         0.06      19.031122  17.232286
 12.631584  20.010368  13.486182  11.07766   16.463406   0.
 17.160105   0.        16.847889   0.         1.       ]

    >>      {'s1': 8.299982} {'j1': 2.2490938} {'p1': 0.0}
    10.449076080322266


 {'M': 0, 'B': 0, 'P': 0, 'Q': 0, 'reg': 0, 'n_violations': 0}
[5.444898  0.        3.9590578 0.       ]


    {'source_blend': {'s1': {'j1': 5.444898}}, 'blend_demand': {'j1': {'p1': 0.0}}, 't

In [253]:
# 0 (only once per episode)
episode_rewards = []
obs = env.reset()
obs, obs_dict = obs

In [262]:
# 1 Get first action
print(env.t)
action, _ = model.predict(obs, deterministic=True)

2


In [263]:
print(env.t)
d = reconstruct_dict(obs, env.mapping_obs)
print(d["sources"])
print(d["blenders"])
print(d["demands"])
print(d["properties"])

2
{'s1': 17.46205}
{'j1': 0.0}
{'p1': 0.0}
{'j1': {'q1': 0.0}}


In [264]:
# 2 Visualize action
print(env.t)
reconstruct_dict(action, env.mapping_act)

2


{'source_blend': {'s1': {'j1': 0.0}},
 'blend_demand': {'j1': {'p1': 30.307917}},
 'tau': {'s1': 8.731916},
 'delta': {'p1': 17.08481}}

In [265]:
# 3
# Step once: get 2nd action
print(env.t)
obs, reward, done, term, _ = env.step(action)

2


In [207]:
# 4 Visualize new state
print(env.t)
d = reconstruct_dict(obs, env.mapping_obs)
print(d["sources"])
print(d["blenders"])
print(d["demands"])
print(d["properties"])

3
{'s1': 26.193966}
{'j1': 0.0}
{'p1': 0.0}
{'j1': {'q1': 0.0}}
